In [1]:
#from rl import RL, train
#from data import DataLoader, make_data_loaders
#from rl import RL, DQNExperiment, train
import os, yaml
import numpy as np
import torch

if os.getcwd()[-4:] == "code":
    os.chdir('../')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import timedelta
import os, math, torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pyprind

In [3]:
with open(os.path.join("./code/params.yaml")) as f:
        params = yaml.safe_load(f)

# run()

In [34]:
np.random.seed(params['random_seed'])
torch.manual_seed(params['random_seed'])
rng = np.random.RandomState(params['random_seed'])

loader_train, loader_validation = make_data_loaders(params['train'], params['val'], rng, params['device'],params['minibatch_size'],params['rolling_size'],type='discharge')


  0%|          | 0/11569 [00:00<?, ?it/s]

  2%|▏         | 258/11569 [00:03<02:29, 75.88it/s]


KeyboardInterrupt: 

In [42]:
train(params, rng, loader_train, loader_validation)

Initializing Experiment
Running experiment (training Q-Networks)

Minibatch learning within epoch


q.shape: torch.Size([64, 11])
a.shape: torch.Size([64, 1])


C:\Users\DAHS\AppData\Local\Temp\ipykernel_197636\99138351.py:124: UserWarning: Using a target size (torch.Size([64, 64])) that is different to the input size (torch.Size([64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.smooth_l1_loss(q_pred, bellman_target)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import timedelta
import os, math, torch
import torch.nn as nn
import torch.nn.functional as F
import operator

class DataLoader(object):
    def __init__(self, df, rng, device, minibatch_size, rolling_size, type):
        self.df = pd.read_parquet(df)
        self.dict = {}
        self.rng = rng
        self.device = device
        self.minibatch_size = minibatch_size
        self.rolling_size = rolling_size
        self.type = type

        self.transition = {}
        self.index_transition = None
        self.index_pos_last = []
        self.index_neg_last = []
        self.data_size = None

        self.transitions_head = None
        self.epoch_finished = True  # to enforce reset() before use
        self.num_minibatches_epoch = None

        self.drop_smaller_than_minibatch = False

    def reset(self, shuffle):
        if shuffle:
            self.rng.shuffle(self.index_transition)
        self.transitions_head = 0
        self.epoch_finished = False
        self.num_minibatches_epoch = int(np.floor(self.data_size / self.minibatch_size)) + int(1 - self.drop_smaller_than_minibatch)

    def make_transition(self):
        self.dict['traj'] = {}
        self.transition['s'] = {}
        self.transition['a'] = {}
        self.transition['r'] = {}
        self.transition['next_s'] = {}
        self.transition['terminal'] = {}
        self.transition['pos_traj'] = []
        self.transition['neg_traj'] = []
        count = 0
        s = [x for x in self.df.columns if x[:2]=='s:']
        a = [x for x in self.df.columns if x[:2]=='a:']
        r = [x for x in self.df.columns if x==('r:'+self.type)]

        for traj in tqdm(self.df.traj.unique()):
            df_i = self.df[self.df['traj']==traj].sort_values(by='step')
            self.dict['traj'][traj] = {'s': [], 'a': [], 'r': []}
            self.dict['traj'][traj]['s'] = df_i[s]
            self.dict['traj'][traj]['a'] = df_i[a]
            self.dict['traj'][traj]['r'] = df_i[r]
            if sum(df_i[r].values) > 0:
                self.transition['pos_traj'].append(traj)
            else :
                self.transition['neg_traj'].append(traj)

            t_len = len(self.dict['traj'][traj]['s']) - self.rolling_size - 1
            for t in range(t_len):
                self.transition['s'][count] = self.dict['traj'][traj]['s'][t:t+self.rolling_size].values.flatten()
                self.transition['a'][count] = self.dict['traj'][traj]['a'][t+self.rolling_size-1:t+self.rolling_size].values.flatten()
                self.transition['r'][count] = self.dict['traj'][traj]['r'][t+self.rolling_size-1:t+self.rolling_size].values.flatten()
                self.transition['next_s'][count] = self.dict['traj'][traj]['s'][t+1:t+self.rolling_size+1].values.flatten()
                self.transition['terminal'][count] = 0
                count += 1
            tlast = t_len
            self.transition['s'][count] = self.dict['traj'][traj]['s'][tlast:tlast+self.rolling_size].values.flatten()
            self.transition['a'][count] = self.dict['traj'][traj]['a'][tlast+self.rolling_size-1:tlast+self.rolling_size].values.flatten()
            self.transition['r'][count] = self.dict['traj'][traj]['r'][tlast+self.rolling_size-1:tlast+self.rolling_size].values.flatten()
            self.transition['next_s'][count] = self.dict['traj'][traj]['s'][tlast+1:tlast+self.rolling_size+1].values.flatten()
            self.transition['terminal'][count] = 1
            #if traj in self.encoded_data['pos_traj']:self.pos_last.append(count)
            #else : self.neg_last.append(count)
            count += 1
        self.data_size = count
        self.index_transition = np.arange(self.data_size)

    def get_next_minibatch(self):
        if self.epoch_finished == True:
            print('Epoch finished, please call reset() method before next call to get_next_minibatch()')
            return None
        # Getting data from dictionaries
        minibatch_main_index_list = list(self.index_transition[self.transitions_head:self.transitions_head + self.minibatch_size])

        minibatch_index_list = minibatch_main_index_list
        get_from_dict = operator.itemgetter(*minibatch_index_list)
        s_minibatch = get_from_dict(self.transition['s'])
        actions_minibatch = get_from_dict(self.transition['a'])
        rewards_minibatch = get_from_dict(self.transition['r'])
        next_s_minibatch = get_from_dict(self.transition['next_s'])
        terminals_minibatch = get_from_dict(self.transition['terminal'])
        # Updating current data head
        self.transitions_head += self.minibatch_size
        self.epoch_finished = self.transitions_head + self.drop_smaller_than_minibatch*self.minibatch_size >= self.data_size
        return s_minibatch, actions_minibatch, rewards_minibatch, next_s_minibatch, terminals_minibatch, self.epoch_finished


def make_data_loaders(train_data, validation_data, rng, device, minibatch_size, rolling_size, type):
    # Note that the loaders will be reset in Experiment
    loader_train = DataLoader(train_data, rng, device, minibatch_size, rolling_size, type)
    loader_validation = DataLoader(validation_data, rng, device, minibatch_size, rolling_size, type)
    loader_train.make_transition()
    loader_validation.make_transition()
    return loader_train, loader_validation

# RL

In [38]:
def init_weights(m):
    if type(m) in [nn.Linear, nn.Conv2d]:
        torch.nn.init.kaiming_uniform_(m.weight)
        m.bias.data.fill_(0.)

class QNetwork(nn.Module):
    def __init__(self, state_dim, rolling_size, nb_actions=None):
        super(QNetwork, self).__init__()

        self.state_dim = state_dim * rolling_size
        self.nb_actions = nb_actions
        
        self.fc = nn.Sequential(
            nn.Linear(self.state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, self.nb_actions)
        )
        self.fc.apply(init_weights)
        
    def forward(self, x):
        x = self.fc(x)
        return x

class IMVFullLSTM(torch.jit.ScriptModule):
    __constants__ = ["n_units", "state_dim"]
    def __init__(self, state_dim, nb_actions, n_units, init_std=0.02):
        super().__init__()
        self.U_j = nn.Parameter(torch.randn(state_dim, 1, n_units)*init_std)
        self.W_j = nn.Parameter(torch.randn(state_dim, n_units, n_units)*init_std)
        self.b_j = nn.Parameter(torch.randn(state_dim, n_units)*init_std)
        self.W_i = nn.Linear(state_dim*(n_units+1), state_dim*n_units)
        self.W_f = nn.Linear(state_dim*(n_units+1), state_dim*n_units)
        self.W_o = nn.Linear(state_dim*(n_units+1), state_dim*n_units)
        self.F_alpha_n = nn.Parameter(torch.randn(state_dim, n_units, 1)*init_std)
        self.F_alpha_n_b = nn.Parameter(torch.randn(state_dim, 1)*init_std)
        self.F_beta = nn.Linear(2*n_units, 1)
        self.Phi = nn.Linear(2*n_units, nb_actions)
        self.n_units = n_units
        self.state_dim = state_dim
    @torch.jit.script_method
    def forward(self, x):
        h_tilda_t = torch.zeros(x.shape[0], self.state_dim, self.n_units).cpu()
        c_t = torch.zeros(x.shape[0], self.state_dim*self.n_units).cpu()
        outputs = torch.jit.annotate(List[Tensor], [])
        for t in range(x.shape[1]):
            # eq 1
            j_tilda_t = torch.tanh(torch.einsum("bij,ijk->bik", h_tilda_t, self.W_j) + \
                                   torch.einsum("bij,jik->bjk", x[:,t,:].unsqueeze(1), self.U_j) + self.b_j)
            inp =  torch.cat([x[:, t, :], h_tilda_t.view(h_tilda_t.shape[0], -1)], dim=1)
            # eq 2
            i_t = torch.sigmoid(self.W_i(inp))
            f_t = torch.sigmoid(self.W_f(inp))
            o_t = torch.sigmoid(self.W_o(inp))
            # eq 3
            c_t = c_t*f_t + i_t*j_tilda_t.reshape(j_tilda_t.shape[0], -1)
            # eq 4
            h_tilda_t = (o_t*torch.tanh(c_t)).view(h_tilda_t.shape[0], self.state_dim, self.n_units)
            outputs += [h_tilda_t]
        outputs = torch.stack(outputs)
        outputs = outputs.permute(1, 0, 2, 3)
        # eq 8
        alphas = torch.tanh(torch.einsum("btij,ijk->btik", outputs, self.F_alpha_n) +self.F_alpha_n_b)
        alphas = torch.exp(alphas)
        #print('alpha.shape:',alphas.shape)
        alphas = alphas/torch.sum(alphas, dim=1, keepdim=True)
        #print('alpha2.shape:',alphas.shape)
        g_n = torch.sum(alphas*outputs, dim=1)
        #print('g_n.shape:',g_n.shape)
        hg = torch.cat([g_n, h_tilda_t], dim=2)
        #print('hg.shape:',hg.shape)
        mu = self.Phi(hg)
        #print('mu.shape:',mu.shape)
        betas = torch.tanh(self.F_beta(hg))
        betas = torch.exp(betas)
        #print('betas.shape:',betas.shape)
        betas = betas/torch.sum(betas, dim=1, keepdim=True)
        #print('betas2.shape:',betas.shape)
        mean = torch.sum(betas*mu, dim=1)
        #print('mean.shape:',mean.shape)
        return mean#, alphas, betas

class RL(object):
    def __init__(self, state_dim, rolling_size, nb_actions, n_units, gamma,
                 learning_rate, update_freq, rng, device):
        self.rng = rng
        self.state_dim = state_dim
        self.rolling_size = rolling_size
        self.nb_actions = nb_actions
        self.n_units = n_units
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.update_freq = update_freq
        self.update_counter = 0
        self.device = device
        self.network = QNetwork(state_dim=self.state_dim, rolling_size=self.rolling_size, nb_actions=self.nb_actions)
        self.target_network = QNetwork(state_dim=self.state_dim, rolling_size=self.rolling_size, nb_actions=self.nb_actions)
        #self.network = IMVFullLSTM(state_dim=self.state_dim, nb_actions=self.nb_actions, n_units=self.n_units)
        #self.target_network = IMVFullLSTM(state_dim=self.state_dim, nb_actions=self.nb_actions, n_units=self.n_units)
        #self.weight_transfer(from_model=self.network, to_model=self.target_network)
        self.network.to(self.device)
        self.target_network.to(self.device)
        self.optimizer = optim.Adam(self.network.parameters(), lr=self.learning_rate, amsgrad=True)

    def train_on_batch(self, s, a, r, s2, t):
        s  = torch.FloatTensor(np.float32(s)).to(self.device)
        s2 = torch.FloatTensor(np.float32(s2)).to(self.device)
        a  = torch.LongTensor(np.int64(a)).to(self.device)
        r  = torch.FloatTensor(np.float32(r)).to(self.device)
        t  = torch.FloatTensor(np.float32(t)).to(self.device)

        with torch.no_grad():
            q = self.network(s).detach()
            q2 = self.target_network(s2).detach()
        print("q.shape:",q.shape)
        print("a.shape:",a.shape)

        q_pred = q.gather(1, a).squeeze(1)
        #q_pred = q.gather(1, a.unsqueeze(1)).squeeze(1)
        
        q2_net = self.network(s2).detach()
        q2_max = q2.gather(1, torch.max(q2_net, 1)[1].unsqueeze(1)).squeeze(1)
        bellman_target = torch.clamp(r, max=1.0, min=0.0) + self.gamma * torch.clamp(q2_max.detach(), max=1.0, min=0.0) * (1 - t)

        loss = F.smooth_l1_loss(q_pred, bellman_target)            
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.detach().cpu().numpy()

    def get_loss(self, s, a, r, s2, t):
        s  = torch.FloatTensor(np.float32(s)).to(self.device)
        s2 = torch.FloatTensor(np.float32(s2)).to(self.device)
        a  = torch.LongTensor(np.int64(a)).to(self.device)
        r  = torch.FloatTensor(np.float32(r)).to(self.device)
        t  = torch.FloatTensor(np.float32(t)).to(self.device)

        with torch.no_grad():
            q = self.network(s).detach()
            q2 = self.target_network(s2).detach()
        print("q:",q.shape,q)
        q_pred = q.gather(1, a).squeeze(1)
        print("q_pred",q_pred.shape,q_pred)
        #q_pred = q.gather(1, a.unsqueeze(1)).squeeze(1)
        
        q2_net = self.network(s2).detach()
        q2_max = q2.gather(1, torch.max(q2_net, 1)[1].unsqueeze(1)).squeeze(1)
        bellman_target = torch.clamp(r, max=1.0, min=0.0) + self.gamma * torch.clamp(q2_max.detach(), max=1.0, min=0.0) * (1 - t)
        print("bellman_target:",bellman_target.shape,bellman_target)
        loss = F.smooth_l1_loss(q_pred, bellman_target)
        return loss.detach().cpu().numpy()

    def get_q(self, s):
        s = torch.FloatTensor(s).to(self.device)
        return self.network(s).detach().cpu().numpy()

    def get_max_action(self, s):
        s = torch.FloatTensor(s).to(self.device)
        q = self.network(s).detach()
        return q.max(1)[1].cpu().numpy()

    def get_action(self, states):
        return self.get_max_action(states)

    def learn(self, s, a, r, s2, term):
        """ Learning from one minibatch """
        loss = self.train_on_batch(s, a, r, s2, term)
        if self.update_counter == self.update_freq:
            self.target_network.load_state_dict(self.network)
            #self.weight_transfer(self.network, self.target_network)
            self.update_counter = 0
        else:
            self.update_counter += 1
        return loss

    # weight_transfer(from_model, to_model):
        #to_model.load_state_dict(from_model.state_dict())


class DQNExperiment(object):
    def __init__(self, data_loader_train, data_loader_validation, q_network, saving_period, rng):
        self.rng = rng
        self.data_loader_train = data_loader_train
        self.data_loader_validation = data_loader_validation
        self.q_network = q_network
        self.batch_num = 0
        self.saving_period = saving_period  # after each `saving_period` epochs, the results so far will be saved.
        '''storage_path = os.path.join(os.path.abspath(folder_location), folder_name)
        self.storage_rl = os.path.join(storage_path, 'rl_' + self.q_network.sided_Q)
        self.checkpoint_folder = os.path.join(storage_path, 'rl_' + self.q_network.sided_Q + '_checkpoints')
        if not os.path.exists(self.storage_rl):
            os.mkdir(self.storage_rl)
        if not os.path.exists(self.checkpoint_folder):
            os.mkdir(self.checkpoint_folder)'''
        
    def do_epochs(self, number):
        '''
        Each epoch is one iteration thorugh the entire dataset.
        '''
        self.curr_epoch = 0
        self.all_epoch_steps = []
        self.all_epoch_validation_steps = []
        self.all_epoch_loss = []
        self.all_epoch_validation_loss = []
        self.data_loader_train.reset(shuffle=True)
        self.data_loader_validation.reset(shuffle=False)
        for epoch in range(self.curr_epoch, number):
            print()
            #print('>>>>> Experiment ' + 'Q-' + self.q_network.sided_Q + ' Epoch ' + str(epoch + 1) + '/' + str(number))
            # Learn here
            epoch_done = False
            epoch_steps = 0
            epoch_loss = 0
            print('Minibatch learning within epoch')
            bar = pyprind.ProgBar(self.data_loader_train.num_minibatches_epoch)
            while not epoch_done:
                s, actions, rewards, next_s, terminals, epoch_done = self.data_loader_train.get_next_minibatch()
                epoch_steps += len(s)
                loss = self.q_network.learn(s, actions, rewards, next_s, terminals)
                epoch_loss += loss
            self.data_loader_train.reset(shuffle=True)
            self.data_loader_validation.reset(shuffle=False, pos_samples_in_minibatch=0, neg_samples_in_minibatch=0)
            self.all_epoch_loss.append(epoch_loss/epoch_steps)
            self.all_epoch_steps.append(epoch_steps)
            if (epoch + 1)% self.saving_period == 0:
                self._do_eval()
                '''try:
                    torch.save({
                        'epoch': epoch,
                        'rl_network_state_dict': self.q_network.network.state_dict(),
                        # 'rl_target_network_state_dict': self.q_network.target_network.state_dict(),
                        # 'rl_optimizer_state_dict': self.q_network.optimizer.state_dict(),
                        'loss': self.all_epoch_loss,
                        'validation_loss': self.all_epoch_validation_loss,
                        'epoch_steps': self.all_epoch_steps,
                        'epoch_validation_steps': self.all_epoch_validation_steps,
                    }, os.path.join(self.checkpoint_folder, 'checkpoint' + str(epoch) +'.pt'))
                    #np.save(os.path.join(self.storage_rl, 'q_losses.npy'), np.array(self.all_epoch_loss))
                except:
                    print(">>> Cannot save files. On Windows: the files might be open.")'''
        
    def _do_eval(self):
        epoch_val_steps = 0
        epoch_val_loss = 0
        epoch_done = False
        bar = pyprind.ProgBar(self.data_loader_validation.num_minibatches_epoch)
        while not epoch_done:
            bar.update()
            s, actions, rewards, next_s, terminals, epoch_done = self.data_loader_validation.get_next_minibatch()
            epoch_val_steps += len(s)
            loss = self.q_network.get_loss(s, actions, rewards, next_s, terminals)
            epoch_val_loss += loss
        self.all_epoch_validation_loss.append(epoch_val_loss / epoch_val_steps)
        self.all_epoch_validation_steps.append(epoch_val_steps)
        '''try:
            np.save(os.path.join(self.storage_rl, 'q_validation_losses.npy'), np.array(self.all_epoch_validation_loss))
        except:
            pass'''

    

def train(params, rng, loader_train, loader_validation):
    qnet = RL(state_dim=params["state_dim"], rolling_size=params["rolling_size"], nb_actions=params["num_actions"], n_units=params["n_units"], gamma=params["gamma"], 
    learning_rate=params["learning_rate"],update_freq=params["update_freq"], rng=params["random_seed"], device=params["device"])
    print('Initializing Experiment')
    expt = DQNExperiment(data_loader_train=loader_train, data_loader_validation=loader_validation, q_network=qnet,
                        saving_period=params["exp_saving_period"], rng=rng)
    '''with open(os.path.join(expt.storage_rl, 'config_exp.yaml'), 'w') as y:
            yaml.safe_dump(params, y)  # saving new params for future reference'''
    print('Running experiment (training Q-Networks)')
    expt.do_epochs(number=params["exp_num_epochs"])
    print("Training Q-Networks finished successfully")

In [ ]:
def run():
    np.random.seed(params['random_seed'])
    torch.manual_seed(params['random_seed'])
    rng = np.random.RandomState(params['random_seed'])

    loader_train, loader_validation = make_data_loaders(params['train'], params['val'], rng, params['device'],params['minibatch_size'],params['rolling_size'],type='discharge')
    train(params, rng, loader_train, loader_validation)

if __name__=='__main__':
    run()